# Scheduler Local Test


## Setup


* `make start-scheduler` 
* `make start-envoy`
* `make start-agent`
* `make start-mlserver`
* `make start-rclone`

## No Auth Example

In [116]:
!grpcurl -d '{"model":{"name":"iris","version":"2","uri":"gs://seldon-models/mlserver/iris",\
         "requirements":["sklearn"],\
         "memoryBytes":500,\
         "replicas":1}}' \
         -plaintext \
         -proto ../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/LoadModel

{
  
}


In [117]:
!grpcurl -d '{"name":"mlserver"}' \
         -plaintext \
         -proto ../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/ServerStatus

{
  "serverName": "mlserver",
  "resources": [
    {
      "memory": "1000",
      "availableMemoryBytes": "500"
    }
  ]
}


In [118]:
!curl -v http://0.0.0.0:9000/v2/models/iris/infer -H "Content-Type: application/json" -H "Host: iris"\
        -d '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}'

* Expire in 0 ms for 6 (transfer 0x55788a5614f0)
*   Trying 0.0.0.0...
* TCP_NODELAY set
* Expire in 200 ms for 4 (transfer 0x55788a5614f0)
* Connected to 0.0.0.0 (127.0.0.1) port 9000 (#0)
> POST /v2/models/iris/infer HTTP/1.1
> Host: iris
> User-Agent: curl/7.64.0
> Accept: */*
> Content-Type: application/json
> Content-Length: 94
> 
* upload completely sent off: 94 out of 94 bytes
< HTTP/1.1 200 OK
< date: Thu, 25 Nov 2021 14:21:40 GMT
< server: envoy
< content-length: 199
< content-type: application/json
< x-envoy-upstream-service-time: 1470
< 
* Connection #0 to host 0.0.0.0 left intact
{"model_name":"iris","model_version":"v0.1.0","id":"099dec52-c1d4-4eb8-a130-747761fabb66","parameters":null,"outputs":[{"name":"predict","shape":[1],"datatype":"INT64","parameters":null,"data":[2]}]}

In [119]:
!cd ../v2 && \
    grpcurl -d '{"model_name":"iris","inputs":[{"name":"input","contents":{"fp32_contents":[1,2,3,4]},"datatype":"FP32","shape":[1,4]}]}' \
        -plaintext \
        -proto grpc_service.proto \
        -rpc-header Seldon:iris \
        0.0.0.0:9000 inference.GRPCInferenceService/ModelInfer

{
  "modelName": "iris",
  "modelVersion": "v0.1.0",
  "outputs": [
    {
      "name": "predict",
      "datatype": "INT64",
      "shape": [
        "1"
      ],
      "contents": {
        "int64Contents": [
          "2"
        ]
      }
    }
  ]
}


In [120]:
!grpcurl -d '{"name":"iris"}' \
         -plaintext \
         -proto ../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/ModelStatus

{
  "modelName": "iris",
  "version": "2",
  "serverName": "mlserver",
  "modelReplicaState": {
    "0": "Loaded"
  }
}


In [121]:
!grpcurl -d '{"name":"mlserver"}' \
         -plaintext \
         -proto ../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/ServerStatus

{
  "serverName": "mlserver",
  "resources": [
    {
      "memory": "1000",
      "availableMemoryBytes": "500"
    }
  ]
}


In [122]:
!grpcurl -d '{"name":"iris"}' \
         -plaintext \
         -proto ../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/UnloadModel

{
  
}


In [123]:
!grpcurl -d '{"name":"iris"}' \
         -plaintext \
         -proto ../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/ModelStatus

ERROR:
  Code: FailedPrecondition
  Message: Failed to find model iris


In [124]:
!grpcurl -d '{"name":"mlserver"}' \
         -plaintext \
         -proto ../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/ServerStatus

{
  "serverName": "mlserver",
  "resources": [
    {
      "memory": "1000",
      "availableMemoryBytes": "1000"
    }
  ]
}


In [125]:
!curl -v http://0.0.0.0:9000/v2/models/iris/infer -H "Content-Type: application/json" -H "Host: iris"\
        -d '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}'

* Expire in 0 ms for 6 (transfer 0x55e2bbebf4f0)
*   Trying 0.0.0.0...
* TCP_NODELAY set
* Expire in 200 ms for 4 (transfer 0x55e2bbebf4f0)
* Connected to 0.0.0.0 (127.0.0.1) port 9000 (#0)







* upload completely sent off: 94 out of 94 bytes






* Closing connection 0


## Inline RClone Config Example

Before running this 
  * install minio in a kind cluster with Ansible.
  * Update ip address for exposed minio ip address in example below

In [126]:
!grpcurl --format text -d '\
    model { \
     name: "iris" \
     version: "1" \
     uri: "s3://models/iris" \
     storageConfig: { \
           storageRcloneConfig: "{\"type\":\"s3\",\"name\":\"s3\",\"parameters\":{\"provider\":\"minio\",\"env_auth\":\"false\",\"access_key_id\":\"minioadmin\",\"secret_access_key\":\"minioadmin\",\"endpoint\":\"http://172.18.255.1:9000\"}}" \
     } \
     requirements: "sklearn" \
     memoryBytes: 500 \
     replicas: 1}' \
         -plaintext \
         -proto ../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/LoadModel

In [127]:
!grpcurl -d '{"name":"mlserver"}' \
         -plaintext \
         -proto ../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/ServerStatus

{
  "serverName": "mlserver",
  "resources": [
    {
      "memory": "1000",
      "availableMemoryBytes": "500"
    }
  ]
}


In [128]:
!curl -v http://0.0.0.0:9000/v2/models/iris/infer -H "Content-Type: application/json" -H "Host: iris"\
        -d '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}'

* Expire in 0 ms for 6 (transfer 0x55abeecde4f0)
*   Trying 0.0.0.0...
* TCP_NODELAY set
* Expire in 200 ms for 4 (transfer 0x55abeecde4f0)
* Connected to 0.0.0.0 (127.0.0.1) port 9000 (#0)
> POST /v2/models/iris/infer HTTP/1.1
> Host: iris
> User-Agent: curl/7.64.0
> Accept: */*
> Content-Type: application/json
> Content-Length: 94
> 
* upload completely sent off: 94 out of 94 bytes
< HTTP/1.1 200 OK
< date: Thu, 25 Nov 2021 14:21:58 GMT
< server: envoy
< content-length: 199
< content-type: application/json
< x-envoy-upstream-service-time: 1416
< 
* Connection #0 to host 0.0.0.0 left intact
{"model_name":"iris","model_version":"v0.1.0","id":"e1c94456-ed6f-42c5-b8f3-3fd1aee14af7","parameters":null,"outputs":[{"name":"predict","shape":[1],"datatype":"INT64","parameters":null,"data":[2]}]}

In [129]:
!cd ../v2 && \
    grpcurl -d '{"model_name":"iris","inputs":[{"name":"input","contents":{"fp32_contents":[1,2,3,4]},"datatype":"FP32","shape":[1,4]}]}' \
        -plaintext \
        -proto grpc_service.proto \
        -rpc-header Seldon:iris \
        0.0.0.0:9000 inference.GRPCInferenceService/ModelInfer

{
  "modelName": "iris",
  "modelVersion": "v0.1.0",
  "outputs": [
    {
      "name": "predict",
      "datatype": "INT64",
      "shape": [
        "1"
      ],
      "contents": {
        "int64Contents": [
          "2"
        ]
      }
    }
  ]
}


In [130]:
!grpcurl -d '{"name":"iris"}' \
         -plaintext \
         -proto ../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/UnloadModel

{
  
}


## K8S Secret RClone Config Example

 * Update endpoint for minio below

In [131]:
%%writefile minio-secret.yaml
apiVersion: v1
kind: Secret
metadata:
  name: minio-secret
  namespace: seldon-mesh
type: Opaque
stringData:
  s3: |
    type: s3
    name: s3
    parameters:
      provider: minio
      env_auth: false
      access_key_id: minioadmin
      secret_access_key: minioadmin
      endpoint: http://172.18.255.1:9000

Overwriting minio-secret.yaml


In [132]:
!kubectl apply -f minio-secret.yaml

secret/minio-secret configured


In [133]:
!grpcurl --format text -d '\
    model { \
     name: "iris" \
     version: "1" \
     uri: "s3://models/iris" \
     storageConfig: { \
      storageSecretName: "minio-secret" \
      } \
     requirements: "sklearn" \
     memoryBytes: 500 \
     replicas: 1}' \
         -plaintext \
         -proto ../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/LoadModel

In [134]:
!grpcurl -d '{"name":"mlserver"}' \
         -plaintext \
         -proto ../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/ServerStatus

{
  "serverName": "mlserver",
  "resources": [
    {
      "memory": "1000",
      "availableMemoryBytes": "500"
    }
  ]
}


In [135]:
!curl -v http://0.0.0.0:9000/v2/models/iris/infer -H "Content-Type: application/json" -H "Host: iris"\
        -d '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}'

* Expire in 0 ms for 6 (transfer 0x55ffb4ed34f0)
*   Trying 0.0.0.0...
* TCP_NODELAY set
* Expire in 200 ms for 4 (transfer 0x55ffb4ed34f0)
* Connected to 0.0.0.0 (127.0.0.1) port 9000 (#0)
> POST /v2/models/iris/infer HTTP/1.1
> Host: iris
> User-Agent: curl/7.64.0
> Accept: */*
> Content-Type: application/json
> Content-Length: 94
> 
* upload completely sent off: 94 out of 94 bytes
< HTTP/1.1 200 OK
< date: Thu, 25 Nov 2021 14:22:56 GMT
< server: envoy
< content-length: 199
< content-type: application/json
< x-envoy-upstream-service-time: 1270
< 
* Connection #0 to host 0.0.0.0 left intact
{"model_name":"iris","model_version":"v0.1.0","id":"697cab9b-f936-4449-a07e-d914aa9edc57","parameters":null,"outputs":[{"name":"predict","shape":[1],"datatype":"INT64","parameters":null,"data":[2]}]}

In [136]:
!cd ../v2 && \
    grpcurl -d '{"model_name":"iris","inputs":[{"name":"input","contents":{"fp32_contents":[1,2,3,4]},"datatype":"FP32","shape":[1,4]}]}' \
        -plaintext \
        -proto grpc_service.proto \
        -rpc-header Seldon:iris \
        0.0.0.0:9000 inference.GRPCInferenceService/ModelInfer

{
  "modelName": "iris",
  "modelVersion": "v0.1.0",
  "outputs": [
    {
      "name": "predict",
      "datatype": "INT64",
      "shape": [
        "1"
      ],
      "contents": {
        "int64Contents": [
          "2"
        ]
      }
    }
  ]
}


In [137]:
!grpcurl -d '{"name":"iris"}' \
         -plaintext \
         -proto ../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/UnloadModel

{
  
}
